In [ ]:
import json
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification


In [ ]:
# Load intents file
with open('intents2.json') as f:
    intents = json.load(f)

# Convert intents to DataFrame
data = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        data.append((pattern, intent['tag']))

df = pd.DataFrame(data, columns=['text', 'label'])


# New Section

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)


In [ ]:
#Prebuild Model

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['label'].unique()))

def convert_examples_to_tf_dataset(examples, labels, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []
    for example in examples:
        inputs = tokenizer.encode_plus(
            example,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
        )
        input_ids.append(inputs['input_ids'])
        attention_masks.append(inputs['attention_mask'])

    return tf.data.Dataset.from_tensor_slices((
        (input_ids, attention_masks),
        labels
    ))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset = convert_examples_to_tf_dataset(train_texts, train_labels, tokenizer)
test_dataset = convert_examples_to_tf_dataset(test_texts, test_labels, tokenizer)

train_dataset = train_dataset.shuffle(len(train_texts)).batch(16)
test_dataset = test_dataset.batch(16)


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=20, validation_data=test_dataset)


Epoch 1/20


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3/3 [==============================] - 191s 26s/step - loss: 2.0901 - accuracy: 0.1000 - val_loss: 2.0855 - val_accuracy: 0.2727
Epoch 2/20
3/3 [==============================] - 61s 19s/step - loss: 1.9854 - accuracy: 0.2500 - val_loss: 2.0386 - val_accuracy: 0.1818
Epoch 3/20
3/3 [==============================] - 60s 19s/step - loss: 1.8933 - accuracy: 0.3750 - val_loss: 1.9779 - val_accuracy: 0.1818
Epoch 4/20
3/3 [==============================] - 63s 21s/step - loss: 1.7955 - accuracy: 0.5000 - val_loss: 1.9204 - val_accuracy: 0.1818
Epoch 5/20
3/3 [==============================] - 61s 20s/step - loss: 1.7460 - accuracy: 0.3750 - val_loss: 1.8554 - val_accuracy: 0.1818
Epoch 6/20
3/3 [==============================] - 60s 19s/step - loss: 1.5729 - accuracy: 0.5750 - val_loss: 1.8208 - val_accuracy: 0.1818
Epoch 7/20
3/3 [======================

In [ ]:
# Prediction functions
def predict_intent(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_class = np.argmax(logits, axis=1)
    predicted_label = label_encoder.inverse_transform(predicted_class)
    return predicted_label[0]


In [ ]:
def get_response(intent, intents_json):
    for i in intents_json['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])
    return "Sorry, I didn't understand that."


In [ ]:
def chatbot_response(text):
    intent = predict_intent(text)
    response = get_response(intent, intents)
    return response


In [ ]:
# Run the chatbot
print("Start talking with the bot (type 'quit' to stop)!")
while True:
    message = input("")
    if message.lower() in ["quit","exit","bye"]:
        break
    response = chatbot_response(message)
    print(response)


Start talking with the bot (type 'quit' to stop)!
hai
Hi there, how can I help?
what courses are offer there
You can study subjects such as Data Science, AI, Machine Learning, Cloud Computing, Cyber Security, etc.
what about the course fess?
Simplilearn offers a variety of online courses and bootcamps to help you learn new skills.
what are the course fees
You can find the pricing information for each course on our website. We also offer discounts from time to time.
how can i enroll
For support, you can contact our customer service at support@simplilearn.com or call us at 1-800-123-4567.
thats helpful
You're Welcome
nice chatting to you
Good to see you again
nice chatting to you,bye
Bye! Come back again soon.
bye
